<a href="https://www.kaggle.com/code/dhruv836/lora-training?scriptVersionId=287297527" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# Clone the repository and install dependencies
!git clone https://github.com/bmaltais/kohya_ss.git
%cd kohya_ss

# This installation script handles all the heavy lifting
!./setup.sh -n

In [ ]:
import os
import shutil
from glob import glob

# --- CONFIGURATION ---
project_name = "slide_revamp_project"
trigger_word = "slidestyl" 
class_word = "background" 
repeats = 100 

# Use the path you copied from the sidebar (without .zip)
# If you are unsure, just use the root input folder, my script will find the images.
input_dataset_path = "/kaggle/input" 

# --- SETUP DIRECTORIES ---
root_dir = f"/kaggle/working/{project_name}"
img_dir = f"{root_dir}/img/{repeats}_{trigger_word} {class_word}"
reg_dir = f"{root_dir}/reg"
model_dir = f"{root_dir}/model"
log_dir = f"{root_dir}/log"

for d in [img_dir, reg_dir, model_dir, log_dir]:
    os.makedirs(d, exist_ok=True)

# --- SMART COPY SCRIPT ---
print(f"Scanning for images in {input_dataset_path}...")
image_count = 0

# Walk through the entire input directory to find images
for root, dirs, files in os.walk(input_dataset_path):
    for file in files:
        # Check if the file is an image
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            source_path = os.path.join(root, file)
            destination_path = os.path.join(img_dir, file)
            
            # Copy the file
            shutil.copy2(source_path, destination_path)
            image_count += 1

if image_count == 0:
    print("ERROR: No images found! Please check your dataset path.")
else:
    print(f"SUCCESS: Found and moved {image_count} images to: {img_dir}")
    print("You are ready for Cell 4 (Captioning).")

In [ ]:
import os

# 1. Define the path to the problematic file based on your error log
blip_file_path = "/kaggle/working/kohya_ss/sd-scripts/finetune/blip/blip.py"

if os.path.exists(blip_file_path):
    # 2. Read the file
    with open(blip_file_path, "r") as f:
        lines = f.readlines()

    # 3. Rewrite the file, commenting out the strict assertion
    with open(blip_file_path, "w") as f:
        patched = False
        for line in lines:
            # This is the line causing the crash
            if "assert(len(msg.missing_keys)==0)" in line:
                f.write("# " + line) # We add a # to turn it into a comment
                patched = True
            else:
                f.write(line)
    
    if patched:
        print("SUCCESS: Patch applied! The strict check has been disabled.")
    else:
        print("WARNING: Could not find the line to patch. Was it already fixed?")
else:
    print(f"ERROR: Could not find file at {blip_file_path}")

In [ ]:
import os
import subprocess

# --- CONFIGURATION ---
project_name = "slide_revamp_project"
trigger_word = "slidestyl"
img_dir = f"/kaggle/working/{project_name}/img/100_{trigger_word} background"
work_dir = "/kaggle/working/kohya_ss/sd-scripts"
script_path = os.path.join(work_dir, "finetune", "make_captions.py")

# --- RUN CAPTIONING ---
print(f"Starting BLIP captioning...")

command = [
    "python", script_path,
    "--batch_size", "8",
    "--caption_extension", ".txt",
    "--caption_weights", "https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_capfilt_large.pth",
    img_dir
]

result = subprocess.run(command, cwd=work_dir, capture_output=True, text=True)

if result.returncode != 0:
    print("Captioning Failed. Error log:")
    print(result.stderr)
else:
    print("BLIP captioning successful!")

    # --- ADD TRIGGER WORD ---
    print(f"Adding trigger word '{trigger_word}' to all captions...")
    txt_files = [f for f in os.listdir(img_dir) if f.endswith(".txt")]
    
    for file in txt_files:
        file_path = os.path.join(img_dir, file)
        with open(file_path, 'r') as f:
            original_caption = f.read().strip()
        with open(file_path, 'w') as f:
            f.write(f"{trigger_word} style, {original_caption}")
            
    print(f"Processed {len(txt_files)} files.")
    print("Verification (First file):")
    os.system(f'ls "{img_dir}"/*.txt | head -n 1 | xargs cat')

In [ ]:
import shutil
from IPython.display import FileLink

# 1. Define the folder we want to zip (where your images and txt files are)
# (Using the variables from previous cells)
project_name = "slide_revamp_project"
trigger_word = "slidestyl"
source_folder = f"/kaggle/working/{project_name}/img/100_{trigger_word} background"
output_filename = "dataset_with_captions"

# 2. Create the zip file
print(f"Zipping folder: {source_folder}...")
shutil.make_archive(output_filename, 'zip', source_folder)

print("Zip created successfully!")

# 3. Create a clickable download link
FileLink(f"{output_filename}.zip")

In [ ]:
# Start the GUI
!python ./kohya_gui.py --share --headless

In [ ]:
import shutil

shutil.make_archive(
    '/kaggle/working/slide_revamp_project/img/',
    'zip',
    '/kaggle/working/slide_revamp_project/img/'
)



In [ ]:
!ls /kaggle/working/slide_revamp_project


In [ ]:
import os

zip_path = "/kaggle/working/slide_revamp_project/img.zip"
print("Exists:", os.path.exists(zip_path))
print("Size:", os.path.getsize(zip_path) / (1024*1024), "MB")
